<a href="https://colab.research.google.com/github/LD-Shell/Chemical_Engineering/blob/main/vapour_pressure_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement

The vapor pressure of a single pure component can be computed using **Antoine's equation**, which is expressed as:

$$
\log{p} = A - \frac{B}{T - C}
$$

where:
- \( p \) is the vapor pressure of the component (in mmHg),
- \( T \) is the temperature (in °C),
- \( A, B, C \) are component-specific constants obtained from experimental data.

### Variables:
- **\( P \)**: Vapor pressure of the component (mmHg)  
- **\( T \)**: Temperature (°C)  
- **\( A, B, C \)**: Component-specific constants  

In [1]:
import os
import pandas as pd
import numpy as np
# This displays all of the columns, preventing Juptyer from redacting them.
pd.set_option('display.max_columns', None)

In [6]:
from google.colab import drive
drive.mount('/content/drive')  # Mount drive if not already done
data_path = 'drive/My Drive/LD-Shell/SE-ALX/Python/Chemical Engineering/vapour_pressure_calculation'
os.chdir(data_path)  # Change the working directory

Mounted at /content/drive


## Import the Antoine Coefficients Database

In [2]:
# Load the combined CSV file
df_combined = pd.read_csv('Antoine_Coefficients.csv')
df_combined.head(5)

,Formula,Compound Name,A,B,C,TMIN,TMAX
0,H2O,Water,8.07131,1730.630,233.426,1,100.0
1,H2O,Water,8.14019,1810.940,244.485,99,374.0
2,CCL4,carbon-tetrachloride,6.89410,1219.580,227.170,-20,101.0
3,CCL3F,trichlorofluoromethane,6.88430,1043.010,236.860,-33,27.0
4,CCL2F2,dichlorodifluoromethane,6.68619,782.072,235.377,-119,-30.0


### Database preparation

In [3]:
df_combined.tail(5)

,Formula,Compound Name,A,B,C,TMIN,TMAX
694,C21H36,1-phenylpentadecane,7.114,2067.0,122.0,216,403.0
695,C21H42,1-cyclopentylhexadecane,7.103,2048.0,121.0,215,401.0
696,C21H42,1-cyclohexylpentadecane,7.072,2078.0,129.0,213,404.0
697,C22H38,1-phenylhexadecane,7.140,2095.0,114.0,227,415.0
698,C22H44,1-cyclohexylhexadecane,7.074,2099.0,122.0,224,416.0


In [4]:
df_combined.rename(columns={
    'Formula': 'formula',
    'Compound Name': 'compound',
    'TMIN': 'tmin',
    'TMAX': 'tmax'}, inplace=True)
df_combined.head(3)

,formula,compound,A,B,C,tmin,tmax
0,H2O,Water,8.07131,1730.63,233.426,1,100.0
1,H2O,Water,8.14019,1810.94,244.485,99,374.0
2,CCL4,carbon-tetrachloride,6.89410,1219.58,227.170,-20,101.0


In [5]:
df_exact= df_combined.iloc[:699, :7]
df_exact.tail()

,formula,compound,A,B,C,tmin,tmax
694,C21H36,1-phenylpentadecane,7.114,2067.0,122.0,216,403.0
695,C21H42,1-cyclopentylhexadecane,7.103,2048.0,121.0,215,401.0
696,C21H42,1-cyclohexylpentadecane,7.072,2078.0,129.0,213,404.0
697,C22H38,1-phenylhexadecane,7.140,2095.0,114.0,227,415.0
698,C22H44,1-cyclohexylhexadecane,7.074,2099.0,122.0,224,416.0


In [6]:
df_exact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   formula   699 non-null    object 
 1   compound  699 non-null    object 
 2   A         699 non-null    float64
 3   B         699 non-null    float64
 4   C         699 non-null    float64
 5   tmin      699 non-null    int64  
 6   tmax      699 non-null    float64
dtypes: float64(4), int64(1), object(2)
memory usage: 38.4+ KB


In [7]:
df_exact.value_counts()

formula  compound                    A        B         C        tmin  tmax 
H2O      Water                       8.14019  1810.940  244.485  99    374.0    1
C10H14   1,2,3,4-tetramethylbenzene  7.05940  1690.540  199.480  80    236.0    1
         1,2,3,5-tetramethylbenzene  7.07790  1675.430  201.140  75    228.0    1
         1,2,4,5-tetramethylbenzene  7.08000  1672.430  201.430  74    227.0    1
         butylbenzene                6.98317  1577.965  201.378  62    213.0    1
                                                                               ..
C10H22   2,2,3,4-tetramethylhexane   6.82410  1443.600  207.300  41    188.0    1
         2,2,3,3-tetramethylhexane   6.84438  1464.030  209.060  41    190.0    1
C10H20O  decanal                     8.20794  2378.316  237.905  52    209.0    1
C10H20   butylcyclohexane            6.91030  1538.518  200.833  59    211.0    1
         1-decene                    6.96034  1501.872  197.578  54    199.0    1
Name: count, Length: 

# Implement vapour pressure calculation based on user input

In [8]:
def calculate_vapour_pressure(df_exact, formula, temperature):
    """
    Calculates the vapour pressure using the Antoine equation.

    Args:
        df_exact: DataFrame containing the Antoine coefficients (A, B, C) for each formula.
        formula: Chemical formula of the compound.
        temperature: Temperature in Celsius.

    Returns:
        DataFrame containing the calculated vapour pressure, and the Antoine coefficients used.
    """

    # Check if the formula exists in the DataFrame
    if formula not in df_exact['formula'].values:
        raise ValueError(f"Formula '{formula}' not found in the Database, kindly suggest to the devs.")

    # Extract Antoine coefficients for the given formula and temperature range
    row = df_exact[(df_exact['formula'] == formula) &
                   (df_exact['tmin'] <= temperature) &
                   (df_exact['tmax'] >= temperature)]

    # Check if there is exactly one row matching the formula and temperature range
    if len(row) != 1:
        raise ValueError(f"No valid data found for formula '{formula}' at temperature {temperature} °C.")

    A = row['A'].values[0]
    B = row['B'].values[0]
    C = row['C'].values[0]
    tmin = row['tmin'].values[0]
    tmax = row['tmax'].values[0]

    # Calculate vapour pressure using Antoine equation
    vapour_pressure = 10 ** (A - B / (temperature + C))

    # Create a DataFrame to store the results
    results_df = pd.DataFrame({
        'formula': [formula],
        'A': [A],
        'B': [B],
        'C': [C],
        'temperature (°C)': [temperature],
        'vapour_pressure (mmHg)': [vapour_pressure]
    })

    return results_df

formula = input("Enter the formula: ")
temperature = float(input("Enter the temperature (Celsius): "))

try:
    results = calculate_vapour_pressure(df_exact, formula, temperature)
    print(results)
except ValueError as e:
    print(e)

Enter the formula:  H2O
Enter the temperature (Celsius):  183


  formula        A        B        C  temperature (°C)  vapour_pressure (mmHg)
0     H2O  8.14019  1810.94  244.485             183.0             8015.385231
